## cut-rod problem

&nbsp;*Length i* | 1 2 3 4 &nbsp;5 &nbsp;6 &nbsp;&nbsp;7 &nbsp; 8 &nbsp; 9  &nbsp; 10|
<br />
&nbsp;*price &nbsp;p* &nbsp;| 1 5 8 9 10 17 17 20 24 30|

In [3]:
def cut_rod(p,n):
    if n == 0:
        return 0
    q=float('-inf')
    for i in range(n):
        q=max(q,p[i]+cut_rod(p,n-i-1))
    return q

In [10]:
p=[1,5,8,9,10,17,17,20,24,30]
for i in range(1,len(p)+1):
    print(f"The max price for length {i}: ", cut_rod(p,i))

The max price for length 1:  1
The max price for length 2:  5
The max price for length 3:  8
The max price for length 4:  10
The max price for length 5:  13
The max price for length 6:  17
The max price for length 7:  18
The max price for length 8:  22
The max price for length 9:  25
The max price for length 10:  30
